In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from google.colab import drive
from array import array
import pandas as pd
import os
import seaborn as sns
drive.mount('/content/gdrive') # 此處需要登入google帳號
data = pd.read_csv("/content/gdrive/My Drive/thesis/thesisdataset/millan_average13_629_final.csv")
data.shape

In [ ]:
data2 = pd.read_csv("/content/gdrive/My Drive/thesis/thesisdataset/taiwan_average37_83_final.csv")
data2.shape

In [ ]:
data3 = pd.read_csv("/content/gdrive/My Drive/thesis/thesisdataset/finland_average_47_786_final.csv")
data3.shape

In [ ]:
!nvidia-smi

In [ ]:
df = pd.DataFrame(data)
dataset = df.internet.values.astype(float)

In [ ]:
df2 = pd.DataFrame(data2)
dataset2 = df2.load.values.astype(float)

In [ ]:
df3 = pd.DataFrame(data3)
dataset3 = df3.Internet.values.astype(float)

In [ ]:
dataset1000=np.append(dataset2, dataset3)
dataset1000=np.append(dataset1000, dataset)

In [ ]:
dataset1000.shape

In [ ]:
dataset1000

In [ ]:
dataset1000=dataset1000.reshape(-1,1)

In [ ]:
dataset1000.shape

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
from torch.autograd import Variable
from sklearn.preprocessing import MinMaxScaler
def sliding_windows(data, seq_length):
    x = []
    y = []

    for i in range(len(data)-1):
        _x = data[i:(i+seq_length)]
        _y = data[i+seq_length]
        x.append(_x)
        y.append(_y)

    return np.array(x),np.array(y)

sc = MinMaxScaler()
dataset1000 = sc.fit_transform(dataset1000)

seq_length = 1
x, y = sliding_windows(dataset1000, seq_length)
train_size = 1516090
# # test_size = 898761
#train_size = int(len(y) * 0.7)
test_size = len(y) - train_size

dataX = Variable(torch.Tensor(np.array(x)))
dataY = Variable(torch.Tensor(np.array(y)))

trainX = Variable(torch.Tensor(np.array(x[0:train_size])))
trainY = Variable(torch.Tensor(np.array(y[0:train_size])))

testX = Variable(torch.Tensor(np.array(x[train_size:len(x)])))
testY = Variable(torch.Tensor(np.array(y[train_size:len(y)])))

In [ ]:
print(trainX.shape)
print(trainY.shape)
print(testX.shape)
print(testY.shape)

In [ ]:
class LSTM(nn.Module):

    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim,dropout_prob):
        super(LSTM, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim
        
        self.lstm = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True,dropout=dropout_prob
        )
        
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h_0 = Variable(torch.zeros(
            self.layer_dim, x.size(0), self.hidden_dim))
        
        c_0 = Variable(torch.zeros(
            self.layer_dim, x.size(0), self.hidden_dim))
        
        # Propagate input through LSTM
        ula, (h_out, _) = self.lstm(x, (h_0, c_0))
        
        h_out = h_out.view(-1, self.hidden_dim)
        
        out = self.fc(h_out)
        
        return out

In [ ]:

import torch
torch.manual_seed(5)
import torch
import numpy as np
import random

seed=5

random.seed(seed)
np.random.seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

# Remove randomness (may be slower on Tesla GPUs) 
# https://pytorch.org/docs/stable/notes/randomness.html
if seed == 5:
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [ ]:

num_epochs = 1000
learning_rate = 0.01
input_dim = 1
hidden_dim = 2
layer_dim = 1  # ONLY CHANGE IS HERE FROM ONE LAYER TO TWO LAYER
output_dim = 1
dropout = 0.1

lstm = LSTM(input_dim, hidden_dim, layer_dim, output_dim,dropout)

criterion = torch.nn.MSELoss()    # mean-squared error for regression
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate)
#optimizer = torch.optim.SGD(lstm.parameters(), lr=learning_rate)L
# Train the model
Loss=[]
for epoch in range(num_epochs):
    outputs = lstm(trainX)
    optimizer.zero_grad()
    
    # obtain the loss function
    loss = criterion(outputs, trainY)
    running_loss = loss.item()
    loss.backward()
    optimizer.step()
    if epoch % 100 == 0:
      Loss.append(running_loss)
      print("Epoch: %d, loss: %1.5f" % (epoch, loss.item()))
Loss0 = np.array(Loss)

In [ ]:
#4
lstm.eval()              
test_predictpkl = lstm(testX)

test_predictpkl = test_predictpkl.data.numpy()

test_predictpkl = sc.inverse_transform(test_predictpkl)
testY_plotpkl = testY.data.numpy()
testY_plotpkl = sc.inverse_transform(testY_plotpkl)
#plt.axvline(x=train_size, c='r', linestyle='--')
plt.plot(testY_plotpkl,color='blue',label='True Value')
plt.plot(test_predictpkl,color='red',label='Predicted Value')
plt.suptitle('LSTM Internet Traffic Prediction')
leg = plt.legend(loc='best', ncol=1, fancybox=True)
leg.get_frame().set_alpha(0.5)
plt.show()
#maml 10task epoch2000
from math import sqrt

from sklearn.metrics import mean_absolute_error

from sklearn.metrics import mean_squared_error

from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error


print("mean_absolute_error:",mean_absolute_error(testY_plotpkl, test_predictpkl))

print("mean_squared_error:",mean_squared_error(testY_plotpkl,test_predictpkl))

print("rmse:",sqrt(mean_squared_error(testY_plotpkl, test_predictpkl)))

print("r2 score:",r2_score(testY_plotpkl, test_predictpkl))
print("mape_sk", mean_absolute_percentage_error(testY_plotpkl, test_predictpkl))

In [ ]:
import joblib
joblib.dump(lstm,"/content/gdrive/My Drive/re_thesis/re_thesis_test/model_test_2/lstm_0610.pkl")
joblib.dump(lstm,"/content/gdrive/My Drive/re_thesis/re_thesis_test/model_test_2/lstm.pkl")

In [ ]:
import joblib
joblib.dump(lstm,"/content/gdrive/My Drive/re_thesis/re_thesis_test/model_test_2/lstm.pkl")

['/content/gdrive/My Drive/re_thesis/re_thesis_test/model_test_2/lstm.pkl']